In [2]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.1/343.1 kB 2.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 3.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: prettytable
    Found existing installation: prettytable 3.10.0
    Uninstalling prettytable-3.10.0:
      Successfully uninstalled prettytable-3.10.0

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
python3 -m pytest

SyntaxError: invalid syntax (3522361780.py, line 1)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import  broadcast, expr, col
spark = SparkSession.builder \
                    .appName("Jupyter") \
                    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
                    .getOrCreate()

25/01/05 11:25:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
from collections import namedtuple

Actor = namedtuple("Actor", "actorid actor films quality_class current_year")
ActorFilms = namedtuple("ActorFilms", "actor actorid year film votes rating")
ActorCumulative = namedtuple("ActorCumulative", "actorid actor films quality_class is_active current_year")


actors_data = [
        Actor("nm0001128"
              , "Alain Delon"
              , [["Le Cercle Rouge","The Love Mates","Borsalino"],[22796, 236, 3578],[8.0,5.8,7.0],[1970,1970,1970]]
              , "average"
              , 1970),
        Actor("nm0000257"
              , "Alan Alda"
              , [["The Moonshine War","Jenny"],[468, 137],[5.9,5.5],[1970,1970]]
              , "bad"
              , 1970),
        Actor("nm0000273"
              , "Alan Arkin"
              , [["Catch-22"], [22272], [7.0],[1970]]
              , "good"
              , 1970),
        Actor("nm0000869"
              , "Alan Bates"
              , [["Three Sisters"], [364], [6.5],[1970]]
              , "average"
              , 1970)
]
actors_df = spark.createDataFrame(actors_data)
print("PRINT actors_df ", actors_df, actors_df.head())
actors_films_data = [
    ActorFilms("Alain Delon"
          , "nm0001128"
          , 1971
          , "Fantasia Among the Squares"
          , 223
          , 4.5),
    ActorFilms("Alain Delon"
        , "nm0001128"
        , 1971
        , "La veuve Couderc"
        , 1468
        , 6.9),
    ActorFilms("Adam Arkin"
          , "nm0035060"
          , 1971
          , "Made for Each Other"
          , 308
          , 6.3)
]
actors_films_df = spark.createDataFrame(actors_films_data)
print("PRINT actors_films_df ", actors_films_df, actors_films_df.head())


PRINT actors_df  DataFrame[actorid: string, actor: string, films: array<array<string>>, quality_class: string, current_year: bigint] Row(actorid='nm0001128', actor='Alain Delon', films=[['Le Cercle Rouge', 'The Love Mates', 'Borsalino'], ['22796', '236', '3578'], ['8.0', '5.8', '7.0'], ['1970', '1970', '1970']], quality_class='average', current_year=1970)
PRINT actors_films_df  DataFrame[actor: string, actorid: string, year: bigint, film: string, votes: bigint, rating: double] Row(actor='Alain Delon', actorid='nm0001128', year=1971, film='Fantasia Among the Squares', votes=223, rating=4.5)


In [3]:
actors_df.createOrReplaceTempView("actors")
actors_films_df.createOrReplaceTempView("actor_films")

In [4]:
%%sql

SELECT actorid, actor, films, quality_class, current_year
    FROM actors
    WHERE current_year = 1970

actorid,actor,films,quality_class,current_year
nm0001128,Alain Delon,"[['Le Cercle Rouge', 'The Love Mates', 'Borsalino'], ['22796', '236', '3578'], ['8.0', '5.8', '7.0'], ['1970', '1970', '1970']]",average,1970
nm0000257,Alan Alda,"[['The Moonshine War', 'Jenny'], ['468', '137'], ['5.9', '5.5'], ['1970', '1970']]",bad,1970
nm0000273,Alan Arkin,"[['Catch-22'], ['22272'], ['7.0'], ['1970']]",good,1970
nm0000869,Alan Bates,"[['Three Sisters'], ['364'], ['6.5'], ['1970']]",average,1970


In [9]:
%%sql
    
SELECT
        actor,
        actorid,
        year,
        ARRAY(ARRAY_AGG(film), ARRAY_AGG(votes), ARRAY_AGG(rating), ARRAY_AGG(year)) AS films,
        -- ARRAY_AGG(ARRAY(ARRAY(film), ARRAY(votes), ARRAY(rating), ARRAY(year))) AS films,
        CASE 
            WHEN avg(rating) > 8 THEN 'star'
            WHEN avg(rating) > 7 AND avg(rating) <= 8 THEN 'good'
            WHEN avg(rating) > 6 AND avg(rating) <= 7 THEN 'average'
            WHEN avg(rating) <= 6 THEN 'bad'
        END as quality_class,
        TRUE AS is_active  
    FROM actor_films
    WHERE year = 1971
    GROUP BY actor, actorid, year

actor,actorid,year,films,quality_class,is_active
Alain Delon,nm0001128,1971,"[['Fantasia Among the Squares', 'La veuve Couderc'], ['223', '1468'], ['4.5', '6.9'], ['1971', '1971']]",bad,True
Adam Arkin,nm0035060,1971,"[['Made for Each Other'], ['308'], ['6.3'], ['1971']]",average,True


In [16]:
%%sql

    
WITH last_year AS (
    SELECT actorid, actor, films, quality_class, current_year
    FROM actors
    WHERE current_year = 1970
),
this_year AS (
    SELECT
        actor,
        actorid,
        year,
        ARRAY(ARRAY_AGG(film), ARRAY_AGG(votes), ARRAY_AGG(rating), ARRAY_AGG(year)) AS films,
        CASE 
            WHEN avg(rating) > 8 THEN 'star'
            WHEN avg(rating) > 7 AND avg(rating) <= 8 THEN 'good'
            WHEN avg(rating) > 6 AND avg(rating) <= 7 THEN 'average'
            WHEN avg(rating) <= 6 THEN 'bad'
        END as quality_class,
        TRUE AS is_active  
    FROM actor_films
    WHERE year = 1971
    GROUP BY actor, actorid, year
)
SELECT
    COALESCE(t.actorid, y.actorid) AS actorid,
    COALESCE(t.actor, y.actor) AS actor,
    y.films[3] as t1 , 
    t.films[3] as t2, 
    CASE 
        WHEN y.films IS NULL THEN t.films
        WHEN t.films IS NOT NULL THEN ARRAY(concat(y.films[0] , t.films[0]) , concat(y.films[1] , t.films[1]) , concat(y.films[2] , t.films[2]), concat(y.films[3] , t.films[3]))
        ELSE y.films
    END AS films,
    COALESCE(t.quality_class, y.quality_class) AS quality_class,
    COALESCE(t.is_active, FALSE) AS is_active,
    COALESCE(t.year, y.current_year + 1) AS current_year
FROM this_year as t
FULL OUTER JOIN last_year as y 
    ON t.actor = y.actor AND t.actorid = y.actorid
ORDER BY actorid

actorid,actor,t1,t2,films,quality_class,is_active,current_year
nm0000257,Alan Alda,"['1970', '1970']",None,"[['The Moonshine War', 'Jenny'], ['468', '137'], ['5.9', '5.5'], ['1970', '1970']]",bad,False,1971
nm0000273,Alan Arkin,['1970'],None,"[['Catch-22'], ['22272'], ['7.0'], ['1970']]",good,False,1971
nm0000869,Alan Bates,['1970'],None,"[['Three Sisters'], ['364'], ['6.5'], ['1970']]",average,False,1971
nm0001128,Alain Delon,"['1970', '1970', '1970']","['1971', '1971']","[['Le Cercle Rouge', 'The Love Mates', 'Borsalino', 'Fantasia Among the Squares', 'La veuve Couderc'], ['22796', '236', '3578', '223', '1468'], ['8.0', '5.8', '7.0', '4.5', '6.9'], ['1970', '1970', '1970', '1971', '1971']]",bad,True,1971
nm0035060,Adam Arkin,None,['1971'],"[['Made for Each Other'], ['308'], ['6.3'], ['1971']]",average,True,1971


In [ ]:
actual_df = do_actor_cumulative_transformation(spark, actors_df, actors_films_df)
print("PRINT ",actual_df, actual_df.head())

In [ ]:
expected_data = [
    ActorCumulative("nm0001128"
          , "Alain Delon"
          , [["Le Cercle Rouge", "The Love Mates", "Borsalino", "Fantasia Among the Squares", "La veuve Couderc"], [22796, 236, 3578, 223, 1468], [8.0, 5.8, 7.0, 4.5, 6.9],
             [1970, 1970, 1970, 1971, 1971]]
          , "bad"
          , True
          , 1971),
    ActorCumulative("nm0000257"
          , "Alan Alda"
          , [["The Moonshine War", "Jenny"], [468, 137], [5.9, 5.5], [1970, 1970]]
          , "bad"
          , False
          , 1971),
    ActorCumulative("nm0000273"
          , "Alan Arkin"
          , [["Catch-22"], [22272], [7.0], [1970]]
          , "good"
          , False
          , 1971),
    ActorCumulative("nm0000869"
          , "Alan Bates"
          , [["Three Sisters"], [364], [6.5], [1970]]
          , "average"
          , False
          , 1971),
    ActorCumulative("nm0035060"
         , "Adam Arkin"
         , [["Made for Each Other"], [308], [6.3], [1971]]
         , "average"
         , True
         , 1971)
]
expected_df = spark.createDataFrame(expected_data)